In [49]:
import redis

# Connect to Redis
redis_host = 'localhost'  # Update this with your Redis host
redis_port = 6379  # Update this with your Redis port
redis_password = None  # If your Redis instance is password protected, set the password here
redis_db = 0  # Update this with your Redis database number

# Connect to Redis
r = redis.StrictRedis(host=redis_host, port=redis_port, password=redis_password, db=redis_db)


# READ DATA from NoSQL(Redis)

In [50]:
# Read session data from Redis to which customer id

sessions = [
    "ses_8923hjxndi", "ses_kljdnsf90e", "ses_f90sjfskf3",
    "ses_23nsd3fdsj", "ses_dj23knsdfd", "ses_23nksdf39s",
    "ses_dj2j3knd3k", "ses_923nkjdksl", "ses_dlkd90slkn",
    "ses_dj2nklsd92", "ses_32nsdklf92", "ses_dj2kn9df02",
    "ses_dklj2ndfs8", "ses_dk2nkdsf23", "ses_dj23kjndf9",
    "ses_32nds0fjds", "ses_dl3kjn9sdf", "ses_djk32nkd02",
    "ses_dkj9230knd", "ses_dj23nkd02s", "ses_dj2knk3sdf",
    "ses_dj32ndk23s", "ses_dklj23ndk3", "ses_dk32j3nkdf",
    "ses_dklj2ndfk2", "ses_dkj32ndk23", "ses_dkl3nkd23s",
    "ses_dklj32nk23", "ses_dj23knds92", "ses_dklj23nd02"
]

for session in sessions:
    session_data = r.hgetall(session)
    if session_data:
        print(f"Session data for {session}:")
        for field, value in session_data.items():
            print(f"{field.decode('utf-8')}: {value.decode('utf-8')}")
        print()
    else:
        print(f"No data found for session {session}")


Session data for ses_8923hjxndi:
customer_id: C00000001
log_in: 27 minutes
click: 54
screen: Home
total_benefits_used: 42
location: South West
sentiment: 215

Session data for ses_kljdnsf90e:
customer_id: C00000002
log_in: 51 minutes
click: 124
screen: Profile
total_benefits_used: 88
location: North East
sentiment: 378

Session data for ses_f90sjfskf3:
customer_id: C00000003
log_in: 14 minutes
click: 89
screen: Search
total_benefits_used: 17
location: Yorkshire
sentiment: 103

Session data for ses_23nsd3fdsj:
customer_id: C00000004
log_in: 38 minutes
click: 201
screen: Notification
total_benefits_used: 59
location: East Midlands
sentiment: 572

Session data for ses_dj23knsdfd:
customer_id: C00000005
log_in: 63 minutes
click: 55
screen: Favourite
total_benefits_used: 36
location: West Midlands
sentiment: 446

Session data for ses_23nksdf39s:
customer_id: C00000006
log_in: 19 minutes
click: 173
screen: Favourite
total_benefits_used: 72
location: South East
sentiment: 199

Session data fo

In [51]:
# Read the membership_renewal_clicks

redis_client = redis.StrictRedis(host=redis_host, port=redis_port, db=redis_db, password=redis_password)

# Key and member
key = "membership_renewal_clicks"
date = "05/03/2024"

# Get the ZSCORE
zscore = redis_client.zscore(key, date)

# Convert to integer
zscore_value = int(zscore) if zscore is not None else None

# Print the result
print("Membership Renewal Clicks for", date, ":", zscore_value)

Membership Renewal Clicks for 05/03/2024 : 14


# UPDATE VALUES in NoSQL (Redis)

In [52]:
# Iterate through all member IDs
for member_id in range(12001, 12027):
    member_id_str = f"MD{member_id:05d}" 
    
    # Retrieve customer ID and status for the current member ID
    customer_id = r.hget(f"membership:{member_id_str}", "customer_id")
    status = r.hget(f"membership:{member_id_str}", "status")
    
    # Print the retrieved data
    print(f"Member ID: {member_id_str}, Customer ID: {customer_id.decode('utf-8')}, Status: {status.decode('utf-8')}")


Member ID: MD12001, Customer ID: C00000001, Status: Active
Member ID: MD12002, Customer ID: C00000002, Status: Active
Member ID: MD12003, Customer ID: C00000004, Status: Active
Member ID: MD12004, Customer ID: C00000020, Status: Not Active
Member ID: MD12005, Customer ID: C00000030, Status: Active
Member ID: MD12006, Customer ID: C00000017, Status: Active
Member ID: MD12007, Customer ID: C00000039, Status: Active
Member ID: MD12008, Customer ID: C00000005, Status: Active
Member ID: MD12009, Customer ID: C00000050, Status: Active
Member ID: MD12010, Customer ID: C00000045, Status: Active
Member ID: MD12011, Customer ID: C00000013, Status: Active
Member ID: MD12012, Customer ID: C00000042, Status: Not Active
Member ID: MD12013, Customer ID: C00000044, Status: Active
Member ID: MD12014, Customer ID: C00000022, Status: Active
Member ID: MD12015, Customer ID: C00000011, Status: Active
Member ID: MD12016, Customer ID: C00000033, Status: Active
Member ID: MD12017, Customer ID: C00000048, Stat

### We want to update Customer ID: C00000030 and its Status from Not Active to Active & Customer ID: C00000028, Status from Active to Not Active

In [53]:
# Update status for customer ID C00000030 (MD12005) to "Not Active"
r.hset("membership:MD12005", "status", "Active")

# Update status for customer ID C00000028 (MD12028) to "Active"
r.hset("membership:MD12028", "status", "Not Active")

# Confirm the update by retrieving and printing the updated data
print("Updated data:")
print("MD12005:", r.hget("membership:MD12005", "status"))
print("MD12028:", r.hget("membership:MD12028", "status"))

Updated data:
MD12005: b'Active'
MD12028: b'Not Active'


# DELETE VALUES in NoSQL (Redis)

In [54]:
# Define the member IDs to delete
member_ids_to_delete = ['MD12018', 'MD12019']

# Delete each member ID and its associated data
for member_id in member_ids_to_delete:
    r.delete(f"membership:{member_id}")

# Confirm the deletion
for member_id in member_ids_to_delete:
    if r.exists(f"membership:{member_id}"):
        print(f"Failed to delete member ID: {member_id}")
    else:
        print(f"Member ID: {member_id} and its associated data have been deleted from Redis.")

Member ID: MD12018 and its associated data have been deleted from Redis.
Member ID: MD12019 and its associated data have been deleted from Redis.


# CREATE VALUES in NoSQL (Redis)

In [55]:
# Conversion rate data
conversion_data = {
    "21/03/2024": "95%",
    "22/03/2024": "38%",
    "23/03/2024": "71%",
    "24/03/2024": "22%",
    "25/03/2024": "68%",
    "26/03/2024": "46%",
    "27/03/2024": "84%",
    "28/03/2024": "31%",
    "29/03/2024": "75%",
    "30/03/2024": "59%"
}

# Insert data into Redis
for date, rate in conversion_data.items():
    r.set(date, rate)

print("Data inserted into Redis successfully.")

Data inserted into Redis successfully.
